In [ ]:
# PySpark transformations: Silver → Gold (aggregations)

from pyspark.sql.window import Window
from pyspark.sql.functions import avg

# Paths
silver_path = "abfss://silver@energydatalaketest.dfs.core.windows.net/load/"
gold_path   = "abfss://gold@energydatalaketest.dfs.core.windows.net/load/"

# Read from Silver
df_silver = spark.read.format("delta").load(silver_path)

# Create 7-day moving average window (168 hours)
windowSpec = Window.partitionBy("region").orderBy("ts").rowsBetween(-168, 0)

df_gold = (
    df_silver.withColumn("mw_7d_avg", avg("mw").over(windowSpec))
)

# Save as Delta to Gold
df_gold.write.format("delta").mode("overwrite").save(gold_path)
